In [3]:
import asyncio, time, sys
import websockets, json
import numpy as np
import syncmetrics as syncm
from numpy_ringbuffer import RingBuffer

from pythonosc import osc_message_builder
from pythonosc import osc_bundle_builder
from pythonosc import udp_client
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

/Users/williamprimett1/anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# OSC output config
local_ip = 'localhost'
local_port = 12000
output_address = '/diff'
client = udp_client.SimpleUDPClient(local_ip, local_port)

#    Send to Processing/MAX/Etc...
def msg(values):
    client.send_message(output_address, values[0])

#    Send to Processing/MAX/Etc...
def send_osc_bundle(values):
    bundle = osc_bundle_builder.OscBundleBuilder(osc_bundle_builder.IMMEDIATELY)
    msg = osc_message_builder.OscMessageBuilder(address=output_address)
    for arg_to_add in values:
        msg.add_arg(arg_to_add)
    bundle.add_content(msg.build())
    bundle = bundle.build()
    client.send(bundle)

def output_labeled(json_data):
    print(json_data)
    for key, value in json_data:
        output_address = "/results/" + label
        msg = osc_message_builder.OscMessageBuilder(
            address=output_address)
        arg_to_add = value
        msg.add_arg(arg_to_add)
        msg.build()
        self.client.send(msg)
#     bundle = osc_bundle_builder.OscBundleBuilder(
#         osc_bundle_builder.IMMEDIATELY)
    

In [5]:
class Session:
    device_data = json.dumps({})
    uri = 'ws://localhost:9001'
    ws_timeout = 5.0
    num_users = 2

    # to do processing in windoes 
    fs=1000
    buffer_W=150
    win_size=15*fs
    window_A = np.zeros(win_size)
    window_B = np.zeros(win_size)
    no_buffer=int(win_size/buffer_W)
    
#     ema_buffers = RingBuffer(capacity=150, dtype=(np.float, num_users)) 
    Window_A = RingBuffer(capacity=win_size, dtype=(float))
    Window_B = RingBuffer(capacity=win_size, dtype=(float))

    def calculate_features(self):
        device_data = json.loads(self.device_data)
        device_data = [device_data["CH1"], device_data["CH2"]]
        self.Window_A.extendleft(device_data[0])
        self.Window_B.extendleft(device_data[1])
#         if self.Window_A.shape[0] % self.win_size == 0:
        if self.Window_A.shape[0] > self.win_size-1:
            buff_A = np.array(self.Window_A)
            buff_B = np.array(self.Window_B)
        
            #add results here
            results = [device_data[0][0], device_data[1][0],
                          syncm.cos_similarity(buff_A, buff_B).flat[0], 
                          syncm.lin_reg_r_metric(buff_A, buff_B),
                          syncm.correlation_coeff(buff_A, buff_B).flat[0]]
            send_osc_bundle(results)

    async def ServerBIT_recieve(self, uri):
        async with websockets.connect(uri) as websocket:
            while True:
                self.device_data = await websocket.recv()
                self.calculate_features()
                await asyncio.sleep(0.0)

    def run(self):
        uri = self.uri
        print("Serving on {}".format(uri))
        try:
            asyncio.get_event_loop().run_until_complete(
                self.ServerBIT_recieve(uri))
        except Exception as e:
            print(e)
            pass
        finally:
            print("no connection, try again")
            exit

In [6]:
session = Session()
session.run()

Serving on ws://localhost:9001
no connection, try again


KeyboardInterrupt: 

In [ ]:
class Session:
    device_data = json.dumps({})
    uri = 'ws://localhost:9001'
    ws_timeout = 5.0
    num_users = 2

    # to do processing in windoes 
    fs=1000
    buffer_W=150
    win_size=15*fs
    window_A = np.zeros(win_size)
    window_B = np.zeros(win_size)
    no_buffer=int(win_size/buffer_W)
    
    ema_buffers = RingBuffer(capacity=150, dtype=(float, num_users)) 
    feature_buffers = RingBuffer(capacity=win_size, dtype=(float, num_users))

    def calculate_features(self):
        device_data = json.loads(self.device_data)
        device_data = [np.mean(device_data["CH1"]), np.mean(device_data["CH2"])]
        self.ema_buffers.appendleft(device_data)
        sigA = (self.ema_buffers[0][0] - self.ema_buffers[0][-1])
        sigB = (self.ema_buffers[1][0] - self.ema_buffers[1][-1])
        self.feature_buffers.appendleft([sigA, sigB])
#         self.feature_buffers.appendleft([float(device_data["CH1"][0]), float(device_data["CH2"][0])])
#         if self.feature_buffers.shape[0] > self.win_size-1:
#             buff_A = np.array([i[0] for i in np.array(self.feature_buffers)])
#             buff_B = np.array([i[1] for i in np.array(self.feature_buffers)])
        if True:
            result_1=(abs(sigA-sigB)/500.0)
#             result_2=syncm.cos_similarity(buff_A, buff_B).flat[0]
#             result_3=syncm.correlation_coeff(buff_A, buff_B).flat[0]
        
            #add results here
            results = [result_1]
            print(results)
            msg(results)

    async def ServerBIT_recieve(self, uri):
        async with websockets.connect(uri) as websocket:
            while True:
                self.device_data = await websocket.recv()
                self.calculate_features()
                await asyncio.sleep(0.0)

    def run(self):
        uri = self.uri
        print("Serving on {}".format(uri))
        try:
            asyncio.get_event_loop().run_until_complete(
                self.ServerBIT_recieve(uri))
        except Exception as e:
            print(e)
            pass
        finally:
            print("no connection, try again")
            exit